In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
timers = []
timers_import_start = time.time() 
# Trainer (has labels)
#trainer_df = pd.read_csv('../datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
learner_tweets_df = pd.read_csv('datasets/cleaned_OLID.tsv', sep="\t")
timers_import_end = time.time()
print(timers_import_end - timers_import_start)

0.020356416702270508


In [6]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
#learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.8 else 0) # threshold the average values
# learner_tweets_df['subtask_a'] = learner_tweets_df['subtask_a'].map({'OFF': 1, 'NOT': 0})

# learner_tweets_df['labels'] = learner_tweets_df["subtask_a"]
# learner_tweets_df.drop(["subtask_a","subtask_b","subtask_c"], axis=1, inplace=True)
sample_size = 40000
positive_ratio = 0.5

# Select the most confident positive values
#semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.8].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
#semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.3].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

#semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])
semi_tweets_df = learner_tweets_df
semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)

semi_tweets = semi_tweets_df['tweet'].values
semi_labels = semi_tweets_df['label'].values


In [7]:
semi_tweets_df.head(10)

,id,tweet,label
12387,27650,why is john kerry running his mouth again as i...,0
385,52965,gun control anyone disarmhate,0
4896,87438,what manga you reading,0
3971,67140,you are right victoria is on the reverse,0
10394,43102,thank you america is respected again maga ka...,0
12889,25562,they are antifa democrats disguised as gop loo...,0
4504,27304,how do you come up with all these lies you hav...,1
3268,56638,bitch my life was flashing before my eyes,1
2439,49314,lmao what a joke he is,0
4688,42416,looking forward to the right dragging up these...,0


In [8]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)

In [9]:
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification

# Number of epochs

tokenizer = DistilBertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')


# Initialize the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2)






# Set up the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
model.train()
num_epochs = 1
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/2207 [00:00<?, ?it/s]/tmp/ipykernel_47463/692897503.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 2207/2207 [01:15<00:00, 29.31it/s]


In [11]:
model.save_pretrained('models/SOLIDOLID_80k_DISTILBERT_1')